In [1]:
## Darren Liu dl499
#SYSEN6888 Deeplearning Homework 2 Problem 1
#October 13 2023

In [2]:
#preprocessing fruitdata

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy

# Create an ImageDataGenerator instance for data normalization and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,  # 15% of the data will be used for validation
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
)

# Training and validation generators
train_generator = train_datagen.flow_from_directory(
    '/Users/kommanderkimchi/Desktop/Deep Learning/ConvNet HW 2/fruits-360_dataset/fruits-360/Training',
    target_size=(75, 75),
    batch_size=1000,
    class_mode='categorical',
    subset='training',  # Use the training data
    seed=42,
)

validation_generator = train_datagen.flow_from_directory(
    '/Users/kommanderkimchi/Desktop/Deep Learning/ConvNet HW 2/fruits-360_dataset/fruits-360/Training',
    target_size=(75, 75),
    batch_size=1000,
    class_mode='categorical',
    subset='validation',  # Use the validation data
    seed=42,
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    '/Users/kommanderkimchi/Desktop/Deep Learning/ConvNet HW 2/fruits-360_dataset/fruits-360/Test',
    target_size=(75, 75),
    batch_size=1000,
    class_mode='categorical',
    seed=42,
)


2023-10-13 23:40:04.043831: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 57612 images belonging to 131 classes.
Found 10080 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [3]:
#model architecture

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense

# Initialize the Sequential model
model = Sequential()

# Add layers as per the given architecture
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='valid', activation='relu', input_shape=(75, 75, 3), kernel_initializer='glorot_uniform', bias_initializer='zeros'))

model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='valid', activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros'))
model.add(BatchNormalization(momentum=0.99, epsilon=0.001))
model.add(Dropout(rate=0.3, seed=99))  # We'll experiment with this rate later
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros'))
model.add(Dense(131, activation='softmax', kernel_initializer='glorot_uniform', bias_initializer='zeros'))

# Show the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 73, 73, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 71, 71, 128)       73856     
                                                                 
 batch_normalization (BatchN  (None, 71, 71, 128)      512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 71, 71, 128)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 35, 35, 128)      0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 156800)            0

In [ ]:
#model training
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Calculate steps per epoch and validation steps
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20, 
    validation_data=validation_generator,
    validation_steps=validation_steps
)


Epoch 1/20


2023-10-13 23:40:12.013139: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


51/57 [=========================>....] - ETA: 1:11:49 - loss: 2.6630 - accuracy: 0.7029

In [ ]:
# Calculate test steps
test_steps = test_generator.samples // test_generator.batch_size

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy * 100}%")

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()


Discussion and Analysis

Model Accuracy: This will show how the accuracy changes over epochs for both the training and validation sets.
Model Loss: This will show how the loss changes over epochs for both the training and validation sets.
Now, regarding the analysis:

Functions of Dropout Values: Dropout helps in reducing overfitting. The higher the dropout rate, the more regularization is added, but it may also slow down the learning process.

Number of Convolutional Layers: Adding more convolutional layers allows the model to learn more complex features. However, it also increases the risk of overfitting, especially if the dataset is not large enough.

In [ ]:
##skip connect model

from tensorflow.keras.layers import Add, Input
from tensorflow.keras.models import Model

# Define the input layer
input_layer = Input(shape=(75, 75, 3))

# First Convolutional Block
x1 = Conv2D(64, (3, 3), strides=(1, 1), padding='valid', activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros')(input_layer)
x = MaxPooling2D(pool_size=(2, 2), padding='valid')(x1)

# Second Convolutional Block
x2 = Conv2D(128, (3, 3), strides=(1, 1), padding='valid', activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros')(x)
x = BatchNormalization(momentum=0.99, epsilon=0.001)(x2)
x = Dropout(rate=0.3)(x)  # We'll stick with a 0.3 dropout rate for this example
x = MaxPooling2D(pool_size=(2, 2), padding='valid')(x)

# Adding Skip Connection (Add the output of the first Conv layer to the output of the second Conv layer)
x = Add()([x, x1])

# Following the rest of the architecture
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros')(x)
output_layer = Dense(131, activation='softmax', kernel_initializer='glorot_uniform', bias_initializer='zeros')(x)

# Create the model with skip connection
model_with_skip = Model(inputs=input_layer, outputs=output_layer)

# Show the model summary
model_with_skip.summary()


In [ ]:
# Compile the model with skip connection
model_with_skip.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with skip connection
history_with_skip = model_with_skip.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=50,  # You can change this to 20 if it's too time-consuming
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Evaluate the model with skip connection on the test data
test_loss_with_skip, test_accuracy_with_skip = model_with_skip.evaluate(test_generator, steps=test_steps)

print(f"Test Loss with Skip Connection: {test_loss_with_skip}")
print(f"Test Accuracy with Skip Connection: {test_accuracy_with_skip * 100}%")

# Plotting for the model with skip connection
plt.figure(figsize=(12, 6))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history_with_skip.history['accuracy'])
plt.plot(history_with_skip.history['val_accuracy'])
plt.title('Model with Skip Connection - Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history_with_skip.history['loss'])
plt.plot(history_with_skip.history['val_loss'])
plt.title('Model with Skip Connection - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()
